In [1]:
import pandas as pd
from dateutil import parser
# result_df = pd.read_csv("../../data/clean_data.100k.csv")
result_df = pd.read_csv("../../data/clean_data.csv")
display(result_df)

,Unnamed: 0,event_dt,screen,feature,action,device_id,session_id,device_vendor,device_model,device_type,os,age,gender,is_finish,is_churn,node_id
0,0,2025-09-29 10:20:27+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,875e50d28d26088a760866c1125ff124
1,1,2025-09-29 10:21:56+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,875e50d28d26088a760866c1125ff124
2,2,2025-09-29 10:22:01+03:00,Еще,Переход в раздел 'Заявки',Тап на кнопку 'Заявки',339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,5281fb229131fa372bd15589fed81bc7
3,3,2025-09-29 10:23:49+03:00,Еще,Открытие экрана,Переход к экрану,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,1,0,875e50d28d26088a760866c1125ff124
4,4,2025-09-29 10:20:17+03:00,Новая заявка,Выбор квартиры,Тап на квартиру,339,10000000009,Redmi,Redmi Note 12,phone,Android,70.0,Ж,0,0,e9c334108c17d0d6014b73f196b775f4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3360645,3360645,2025-10-09 16:18:31+03:00,Новое ОСС,Открытие экрана,Переход к экрану,123401,10000000010,Apple,iPhone 7 Plus,phone,iOS,51.0,Ж,1,0,18b0d595350748c3fd82491de6631219
3360646,3360646,2025-10-09 18:56:36+03:00,Еще,Открытие экрана,Переход к экрану,152365,10000000054,Apple,iPhone XR,phone,iOS,49.0,Ж,1,0,875e50d28d26088a760866c1125ff124
3360647,3360647,2025-10-09 14:41:10+03:00,Еще,Открытие экрана,Переход к экрану,99472,10000000098,Apple,iPhone 14 Pro Max,phone,iOS,34.0,Ж,0,0,875e50d28d26088a760866c1125ff124
3360648,3360648,2025-10-09 14:41:16+03:00,Еще,Переход в раздел 'Заявки',Тап на кнопку 'Заявки',99472,10000000098,Apple,iPhone 14 Pro Max,phone,iOS,34.0,Ж,0,0,5281fb229131fa372bd15589fed81bc7


## Агрегация нод

Нодой графа будут все уникальные сочетания экран-возможность. `node_id` сформирован как hash от этих полей, поэтому можем опираться на него.

Так же для каждой ноды формируем перечень фич:

**Основные идентификаторы и категории:**
- `screen` – экран/страница, на которой находится нода
- `feature` – функциональный блок/компонент интерфейса
- `action` – действие, которое выполняет нода (например, клик)

**Статистика посещений:**
- `total_visits` – общее количество событий (визитов) на ноде
- `sessions_with_node` – количество сессий, в которых появлялась нода
- `avg_visits_per_session` – среднее количество посещений ноды в рамках одной сессии
- `median_visits_per_session` – медианное количество посещений ноды в сессии
- `repeat_ratio` – доля сессий, в которых нода посещалась более одного раза

**Временные метрики:**
- `avg_time_on_page_seconds` – среднее время до следующего события (приблизительное время на странице)
- `median_time_on_page_seconds` – медианное время до следующего события
- `avg_session_duration_seconds` – средняя длительность сессий, в которых появлялась нода
- `avg_session_length` – средняя длина сессий (количество событий) с участием ноды

**Метрики оттока и завершения:**
- `churn_count` – количество событий с оттоком (is_churn)
- `churn_rate` – доля событий с оттоком от общего числа посещений
- `bounce_count` – количество событий с завершением (is_finish)
- `bounce_rate` – доля событий с завершением от общего числа посещений

**Демографические характеристики:**
- `avg_age` – средний возраст пользователей, взаимодействовавших с нодой
- `male_ratio` – доля мужчин среди пользователей ноды

**Технические характеристики устройства:**
- `device_vendor_top` – наиболее частый производитель устройства
- `device_type_top` – наиболее частый тип устройства
- `os_top` – наиболее частая операционная система

In [2]:
# Полный код функции агрегации нод (node-level features) из event-лога.
#
# Ожидаемые колонки во входном DataFrame `df`:
# - node_id, event_dt, screen, feature, action,
# - device_id, session_id,
# - device_vendor, device_model, device_type, os,
# - age, gender,
# - is_churn, is_finish,
#
# Простой пример использования внизу.

import pandas as pd
import numpy as np
from typing import List

def aggregate_node_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Aggregate node-level features from event-level logs.
    Returns DataFrame indexed by node_id with aggregated features.
    """
    required_cols = {
        'node_id','event_dt','device_id','session_id','age','gender',
        'is_churn','is_finish','screen', 'feature', 'action'
    }
    missing = required_cols.difference(set(df.columns))
    if missing:
        raise ValueError(f"Input df is missing required columns: {missing}")

    df = df.copy()

    # Ensure datetime
    if not isinstance(df['event_dt'].dtype, pd.DatetimeTZDtype):
        df['event_dt'] = pd.to_datetime(df['event_dt'])

    # Compose unique session key
    df['session_key'] = df['device_id'].astype(str) + '||' + df['session_id'].astype(str)

    # Sort by device/session/time so diffs work predictably
    df = df.sort_values(['device_id','session_id','event_dt'])

    # Time to next event within same session (proxy for time on page)
    df['time_to_next'] = df.groupby('session_key')['event_dt'].shift(-1) - df['event_dt']
    df['time_to_next_seconds'] = df['time_to_next'].dt.total_seconds()

    # Session-level aggregates
    sessions = df.groupby('session_key').agg(
        session_start=('event_dt','min'),
        session_end=('event_dt','max'),
        session_length_events=('event_dt','count'),
        device_id=('device_id','first'),
    ).reset_index()
    sessions['session_duration_seconds'] = (sessions['session_end'] - sessions['session_start']).dt.total_seconds()

    # Merge session-level back to events
    df = df.merge(
        sessions[['session_key','session_duration_seconds','session_length_events']],
        on='session_key',
        how='left'
    )

    # Per-node-per-session counts: how many events of this node in the session
    node_session = df.groupby(['node_id','session_key']).agg(
        events_in_session=('event_dt','count'),
    ).reset_index()

    # Node-session level stats
    node_session_stats = node_session.groupby('node_id').agg(
        sessions_with_node=('session_key','nunique'),
        avg_visits_per_session=('events_in_session','mean'),
        median_visits_per_session=('events_in_session','median'),
        repeat_session_count=('events_in_session', lambda s: (s>1).sum())
    ).reset_index()
    node_session_stats['repeat_ratio'] = node_session_stats['repeat_session_count'] / node_session_stats['sessions_with_node']
    node_session_stats = node_session_stats.fillna(0)

    # Node-level aggregations from events
    # For mode calculations create helper that returns NaN-safe mode
    def mode_or_nan(s):
        s = s.dropna().astype(str)
        if s.shape[0] == 0:
            return np.nan
        try:
            return s.mode().iloc[0]
        except Exception:
            return s.iloc[0]

    agg_funcs = {
        'event_dt': 'count',  # total events -> total_visits
        'screen': 'first',
        'feature': 'first', 
        'action': 'first',
        'time_to_next_seconds': ['mean','median'],
        'session_duration_seconds': 'mean', # average session duration for sessions where node appears
        'is_churn': 'sum',
        'is_finish': 'sum',
        'age': 'mean',
        'gender': lambda s: np.mean(s.astype(str).str.lower().isin(['m','male','м','муж'])),
        'device_vendor': lambda s: mode_or_nan(s),
        'device_type': lambda s: mode_or_nan(s),
        'os': lambda s: mode_or_nan(s),
    }
    node_agg = df.groupby('node_id').agg(agg_funcs)
    # flatten MultiIndex columns
    node_agg.columns = ['_'.join(filter(None, col)).strip() for col in node_agg.columns.values]

    # Rename predictable columns
    rename_map = {
        'screen_first': 'screen',
        'feature_first': 'feature', 
        'action_first': 'action',
    }
    if 'event_dt_count' in node_agg.columns:
        rename_map['event_dt_count'] = 'total_visits'
    if 'time_to_next_seconds_mean' in node_agg.columns:
        rename_map['time_to_next_seconds_mean'] = 'avg_time_on_page_seconds'
    if 'time_to_next_seconds_median' in node_agg.columns:
        rename_map['time_to_next_seconds_median'] = 'median_time_on_page_seconds'
    if 'session_duration_seconds_mean' in node_agg.columns:
        rename_map['session_duration_seconds_mean'] = 'avg_session_duration_seconds'
    if 'is_churn_sum' in node_agg.columns:
        rename_map['is_churn_sum'] = 'churn_count'
    if 'is_finish_sum' in node_agg.columns:
        rename_map['is_finish_sum'] = 'bounce_count'
    if 'age_mean' in node_agg.columns:
        rename_map['age_mean'] = 'avg_age'
    # gender lambda may have name like 'gender_<lambda>'
    for col in node_agg.columns:
        if col.startswith('gender_'):
            rename_map[col] = 'male_ratio'
        if col.startswith('device_vendor_'):
            rename_map[col] = 'device_vendor_top'
        if col.startswith('device_type_'):
            rename_map[col] = 'device_type_top'
        if col.startswith('os_'):
            rename_map[col] = 'os_top'

    node_agg = node_agg.rename(columns=rename_map)

    node_agg = node_agg.reset_index()

    # avg_session_length: average length of sessions where node appears
    sessions_where_node = df[['node_id','session_key','session_length_events']].drop_duplicates()
    avg_session_length = sessions_where_node.groupby('node_id')['session_length_events'].mean().reset_index().rename(columns={'session_length_events':'avg_session_length'})
    node_features = node_agg.merge(node_session_stats, on='node_id', how='left').merge(avg_session_length, on='node_id', how='left')

    # churn_rate and bounce_rate relative to total_visits (events)
    # Protect against division by zero
    node_features['churn_rate'] = node_features.apply(lambda r: r['churn_count']/r['total_visits'] if r.get('total_visits',0)>0 else 0, axis=1)
    node_features['bounce_rate'] = node_features.apply(lambda r: r['bounce_count']/r['total_visits'] if r.get('total_visits',0)>0 else 0, axis=1)

    # Fill NaNs for numeric columns
    numeric_cols = node_features.select_dtypes(include=[np.number]).columns.tolist()
    node_features[numeric_cols] = node_features[numeric_cols].fillna(0)

    # For categorical/top-mode columns fill with 'UNKNOWN'
    cat_cols = [c for c in ['device_vendor_top','device_type_top','os_top'] if c in node_features.columns]
    for c in cat_cols:
        node_features[c] = node_features[c].fillna('UNKNOWN')

    # Choose final columns to keep (existing ones)
    keep_cols = [
        'node_id',
        'screen', 'feature', 'action',
        # counts & session stats
        'total_visits','sessions_with_node','avg_visits_per_session','median_visits_per_session','avg_session_length',
        # time stats
        'avg_time_on_page_seconds','median_time_on_page_seconds','avg_session_duration_seconds',
        # repeat
        'repeat_ratio',
        # churn/bounce
        'churn_count','churn_rate','bounce_count','bounce_rate',
        # demographics
        'avg_age','male_ratio',
        # device/os top
        'device_vendor_top','device_type_top','os_top'
    ]
    existing_keep = [c for c in keep_cols if c in node_features.columns]

    result = node_features[existing_keep].set_index('node_id')

    # final numeric fill
    num_cols = result.select_dtypes(include=[np.number]).columns.tolist()
    result[num_cols] = result[num_cols].fillna(0)

    # final string fill
    for c in result.columns:
        if result[c].dtype == object:
            result[c] = result[c].fillna('UNKNOWN')

    return result

In [3]:
df = result_df.copy()
node_df = aggregate_node_features(df)
print(node_df.head().to_string())
node_df.to_csv("../../data/node_df.csv", index=True)

                                           screen                                             feature                                           action  total_visits  sessions_with_node  avg_visits_per_session  median_visits_per_session  avg_session_length  avg_time_on_page_seconds  median_time_on_page_seconds  avg_session_duration_seconds  repeat_ratio  churn_count  churn_rate  bounce_count  bounce_rate    avg_age  male_ratio device_vendor_top device_type_top   os_top
node_id                                                                                                                                                                                                                                                                                                                                                                                                                                                                                 
0072f89b60d46ef6f2094949d8831f13           Важное     

### EDGE FEATURES — агрегирование связей

Под связью считаем переход node_i → node_j в рамках одного device_id + session_id, на основе сортировки по event_dt.

Связи уникальны в итоговом наборе.
Для каждого ребра считаем агрегаты:

- `transition_count` — сколько раз переход был замечен
- `unique_users` — из скольких device_id этот переход
- `unique_sessions` — сколько сессий давали этот переход
- `avg_time_between_nodes_sec` — среднее время между нодами
- `median_time_between_nodes_sec`
- `bounce_rate_on_target` — сколько target-node была bounce
- `churn_rate_on_target` — сколько target-node была churn

In [4]:
# Формирование набора связей для GNN из event-логов.
# Требуются колонки:
# node_id, event_dt, device_id, session_id, is_churn, is_finish

import pandas as pd
import numpy as np

def aggregate_edge_features(df: pd.DataFrame) -> pd.DataFrame:
    """
    Формирует агрегаты по переходам node_i -> node_j.
    Возвращает таблицу с edge features, индекс — (src_node_id, dst_node_id).
    """
    required = {'node_id', 'event_dt', 'device_id', 'session_id', 'is_churn', 'is_finish'}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing columns: {missing}")

    df = df.copy()

    # гарантируем datetime
    if not isinstance(df['event_dt'].dtype, pd.DatetimeTZDtype):
        df['event_dt'] = pd.to_datetime(df['event_dt'])

    # сортировка
    df = df.sort_values(["device_id", "session_id", "event_dt"]).reset_index(drop=True)

    # переходы внутри одной сессии
    df['next_node'] = df.groupby(['device_id', 'session_id'])['node_id'].shift(-1)
    df['next_event_dt'] = df.groupby(['device_id', 'session_id'])['event_dt'].shift(-1)

    # время между нодами
    df['delta_t'] = (df['next_event_dt'] - df['event_dt']).dt.total_seconds()

    # оставляем только строки, где переход существует
    transitions = df.dropna(subset=['next_node']).copy()

    # target node bounce/churn (по next_node)
    transitions['target_node'] = transitions['next_node']

    # Важный момент: is_churn относится к самой ноде, а нам нужно перенести его на target
    transitions = transitions.merge(
        df[['node_id', 'device_id', 'session_id', 'is_churn', 'is_finish',]],
        left_on=['target_node', 'device_id', 'session_id'],
        right_on=['node_id', 'device_id', 'session_id'],
        how='left',
        suffixes=("", "_target")
    )

    # удаляем дубль node_id_target
    transitions = transitions.drop(columns=['node_id_target'])

    # агрегаты на уровне пары нод (src → dst)
    edge = transitions.groupby(['node_id', 'target_node']).agg(
        transition_count=('target_node', 'count'),
        unique_users=('device_id', 'nunique'),
        unique_sessions=('session_id', 'nunique'),
        avg_time_between_nodes_sec=('delta_t', 'mean'),
        median_time_between_nodes_sec=('delta_t', 'median'),
        bounce_rate_on_target=('is_finish_target', 'mean'),
        churn_rate_on_target=('is_churn_target', 'mean'),
    ).reset_index()

    # финальный формат
    edge = edge.rename(columns={
        'node_id': 'src_node',
        'target_node': 'dst_node'
    })

    # сортировка для удобства
    edge = edge.sort_values(['src_node', 'dst_node']).reset_index(drop=True)

    return edge.set_index(['src_node', 'dst_node'])



In [5]:
result_df.to_parquet("events.parquet", index=False)
df = pd.read_parquet("events.parquet", engine="pyarrow")

edge_df = aggregate_edge_features(df)
display(edge_df)
# node_ids = list(node_df.index)
# filtered_edge_df, stats = filter_edges_by_nodes(edge_df, node_ids)
edge_df.to_csv("../../data/edge_df.csv", index=True)

# display(filtered_edge_df.head())

transition_count  \
src_node                         dst_node                                             
0072f89b60d46ef6f2094949d8831f13 0072f89b60d46ef6f2094949d8831f13             13271   
                                 02b207cc24a78c1942161bafc72fe532               290   
                                 0ab7553a46130fe3b64fa66ae66e6ad1                35   
                                 137091633a6334ce94ced79cab6ec771                 5   
                                 18b0d595350748c3fd82491de6631219                23   
...                                                                             ...   
f45c33fe096f325179397402f5b9eb00 f2f9d242858a788cad0cd1e66264f25b                53   
                                 f45c33fe096f325179397402f5b9eb00                67   
                                 f97ca76f771ec1aa83719e1992d1efdd                 1   
f97ca76f771ec1aa83719e1992d1efdd f2f9d242858a788cad0cd1e66264f25b                 1   
                                 f45c33fe096f325179397402f5b9eb00                 2   

                                                                   unique_users  \
src_node                         dst_node                                         
0072f89b60d46ef6f2094949d8831f13 0072f89b60d46ef6f2094949d8831f13          1409   
                                 02b207cc24a78c1942161bafc72fe532           255   
                                 0ab7553a46130fe3b64fa66ae66e6ad1            30   
                                 137091633a6334ce94ced79cab6ec771             5   
                                 18b0d595350748c3fd82491de6631219            15   
...                                                                         ...   
f45c33fe096f325179397402f5b9eb00 f2f9d242858a788cad0cd1e66264f25b            45   
                                 f45c33fe096f325179397402f5b9eb00            26   
                                 f97ca76f771ec1aa83719e1992d1efdd             1   
f97ca76f771ec1aa83719e1992d1efdd f2f9d242858a788cad0cd1e66264f25b             1   
                                 f45c33fe096f325179397402f5b9eb00             1   

                                                                   unique_sessions  \
src_node                         dst_node                                            
0072f89b60d46ef6f2094949d8831f13 0072f89b60d46ef6f2094949d8831f13              353   
                                 02b207cc24a78c1942161bafc72fe532               85   
                                 0ab7553a46130fe3b64fa66ae66e6ad1               28   
                                 137091633a6334ce94ced79cab6ec771                5   
                                 18b0d595350748c3fd82491de6631219               11   
...                                                                            ...   
f45c33fe096f325179397402f5b9eb00 f2f9d242858a788cad0cd1e66264f25b               35   
                                 f45c33fe096f325179397402f5b9eb00               23   
                                 f97ca76f771ec1aa83719e1992d1efdd                1   
f97ca76f771ec1aa83719e1992d1efdd f2f9d242858a788cad0cd1e66264f25b                1   
                                 f45c33fe096f325179397402f5b9eb00                1   

                                                                   avg_time_between_nodes_sec  \
src_node                         dst_node                                                       
0072f89b60d46ef6f2094949d8831f13 0072f89b60d46ef6f2094949d8831f13                   15.936101   
                                 02b207cc24a78c1942161bafc72fe532                   43.103448   
                                 0ab7553a46130fe3b64fa66ae66e6ad1                   46.628571   
                                 137091633a6334ce94ced79cab6ec771                   79.200000   
                                 18b0d595350748c3fd82491de6631219                   84.086957   
...                                                      

### Отток клиентов

In [8]:
import pandas as pd

df = pd.read_csv("../../data/clean_data.csv")

# 1. Приводим дату (таймзона сохранится)
df["event_dt"] = pd.to_datetime(df["event_dt"])

# 2. Берём минимальную дату (tz-aware)
min_dt = df["event_dt"].min()

# 3. Начало первого месяца (tz-aware)
first_month_start = min_dt.replace(
    day=1, hour=0, minute=0, second=0, microsecond=0
)

# 4. Конец периода, для которого churn можно корректно определить
first_month_end = df["event_dt"].max() - pd.Timedelta(days=31)


# 5. Клиенты первого месяца
first_month_users = df.loc[
    (df["event_dt"] >= first_month_start) &
    (df["event_dt"] <= first_month_end),
    "device_id"
].unique()

# 6. Когорта
cohort_df = df[df["device_id"].isin(first_month_users)]

# 7. Churn по клиентам
user_churn = (
    cohort_df
    .groupby("device_id")["is_churn"]
    .any()
)

# 8. Churn-rate
churn_rate = user_churn.mean()

churn_rate


np.float64(0.4953481525099367)

In [9]:
# Количество пользователей
churn_counts = user_churn.value_counts()

churn_users = churn_counts.get(True, 0)
non_churn_users = churn_counts.get(False, 0)

print(f"Churn users: {churn_users}")
print(f"Non-churn users: {non_churn_users}")


Churn users: 67298
Non-churn users: 68562
